In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import PoissonRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from sklearn import linear_model

In [2]:
pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True)

## Load Data
Load both train and test datasets after preprocessing.

In [3]:
df_train = pd.read_csv('data/ift6758-a20/train_clean.csv')

In [4]:
df_test = pd.read_csv('data/ift6758-a20/test_clean.csv')

In [5]:
df_train.head()

,Id,User Name,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location,Location Public Visibility,User Language,Profile Creation Timestamp,User Time Zone,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,Num of Profile Likes,num_profile_likes_log,user_name_length,personal_url_binary,cover_image_binary,verification_status_not verified,verification_status_pending,verification_status_verified,profile_text_color,profile_text_color_aqua,profile_text_color_black,profile_text_color_blue,profile_text_color_fuchsia,profile_text_color_gray,profile_text_color_green,profile_text_color_lime,profile_text_color_maroon,profile_text_color_navy,profile_text_color_olive,profile_text_color_purple,profile_text_color_red,profile_text_color_silver,profile_text_color_teal,profile_text_color_white,profile_text_color_yellow,profile_page_color,profile_page_color_aqua,profile_page_color_black,profile_page_color_blue,profile_page_color_fuchsia,profile_page_color_gray,profile_page_color_green,profile_page_color_lime,profile_page_color_maroon,profile_page_color_navy,profile_page_color_olive,profile_page_color_purple,profile_page_color_red,profile_page_color_silver,profile_page_color_teal,profile_page_color_white,profile_page_color_yellow,profile_theme_color,profile_theme_color_aqua,profile_theme_color_black,profile_theme_color_blue,profile_theme_color_fuchsia,profile_theme_color_gray,profile_theme_color_green,profile_theme_color_lime,profile_theme_color_maroon,profile_theme_color_navy,profile_theme_color_olive,profile_theme_color_purple,profile_theme_color_red,profile_theme_color_silver,profile_theme_color_teal,profile_theme_color_white,profile_theme_color_yellow,profile_view_size_custom,utc_offset,utc_offset__-10800.0,utc_offset__-14400.0,utc_offset__-18000.0,utc_offset__-21600.0,utc_offset__-25200.0,utc_offset__-28800.0,utc_offset__-36000.0,utc_offset__-39600.0,utc_offset__-7200.0,utc_offset__10800.0,utc_offset__14400.0,utc_offset__18000.0,utc_offset__19800.0,utc_offset__25200.0,utc_offset__28800.0,utc_offset__32400.0,utc_offset__3600.0,utc_offset__36000.0,utc_offset__39600.0,utc_offset__46800.0,utc_offset__7200.0,utc_offset__other,utc_offset__unk,location_clean,location_binary,location_public_binary,user_language,user_lang_ar,user_lang_de,user_lang_en,user_lang_en-gb,user_lang_es,user_lang_fr,user_lang_id,user_lang_it,user_lang_ja,user_lang_ko,user_lang_nl,user_lang_other,user_lang_pl,user_lang_pt,user_lang_ru,user_lang_tr,profile_creation_timestamp,profile_creation_year,profile_creation_month,profile_creation_day,user_timezone,user_timezone_abu dhabi,user_timezone_alaska,user_timezone_america/new_york,user_timezone_amsterdam,user_timezone_arizona,user_timezone_athens,user_timezone_atlantic time (canada),user_timezone_baghdad,user_timezone_bangkok,user_timezone_beijing,user_timezone_belgrade,user_timezone_berlin,user_timezone_bern,user_timezone_bogota,user_timezone_brasilia,user_timezone_brussels,user_timezone_buenos aires,user_timezone_cairo,user_timezone_caracas,user_timezone_casablanca,user_timezone_central america,user_timezone_central time (us & canada),user_timezone_chennai,user_timezone_copenhagen,user_timezone_dublin,user_timezone_eastern time (us & canada),user_timezone_greenland,user_timezone_hanoi,user_timezone_hawaii,user_timezone_hong kong,user_timezone_irkutsk,user_timezone_islamabad,user_timezone_istanbul,user_timezone_jakarta,user_timezone_jerusalem,user_timezone_karachi,user_timezone_kuala lumpur,user_timezone_kuwait,user_timezone_kyiv,user_timezone_lima,user_timezone_lisbon,user_timezone_ljubljana,user_timezone_london,user_timezone_madrid,user_timezone_melbourne,user_timezone_mexico city,user_timezone_moscow,user_timezone_mountain time (us & canada),user_timezone_mumbai,user_timezone

In [6]:
df_test.head()

,Id,User Name,Personal URL,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location,Location Public Visibility,User Language,Profile Creation Timestamp,User Time Zone,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image,user_name_length,personal_url_binary,cover_image_binary,verification_status_not verified,verification_status_pending,verification_status_verified,profile_text_color,profile_text_color_aqua,profile_text_color_black,profile_text_color_blue,profile_text_color_fuchsia,profile_text_color_gray,profile_text_color_green,profile_text_color_lime,profile_text_color_maroon,profile_text_color_navy,profile_text_color_olive,profile_text_color_purple,profile_text_color_red,profile_text_color_silver,profile_text_color_teal,profile_text_color_white,profile_text_color_yellow,profile_page_color,profile_page_color_aqua,profile_page_color_black,profile_page_color_blue,profile_page_color_fuchsia,profile_page_color_gray,profile_page_color_green,profile_page_color_lime,profile_page_color_maroon,profile_page_color_navy,profile_page_color_olive,profile_page_color_purple,profile_page_color_red,profile_page_color_silver,profile_page_color_teal,profile_page_color_white,profile_page_color_yellow,profile_theme_color,profile_theme_color_aqua,profile_theme_color_black,profile_theme_color_blue,profile_theme_color_fuchsia,profile_theme_color_gray,profile_theme_color_green,profile_theme_color_lime,profile_theme_color_maroon,profile_theme_color_navy,profile_theme_color_olive,profile_theme_color_purple,profile_theme_color_red,profile_theme_color_silver,profile_theme_color_teal,profile_theme_color_white,profile_theme_color_yellow,profile_view_size_custom,utc_offset,utc_offset__-10800.0,utc_offset__-14400.0,utc_offset__-18000.0,utc_offset__-21600.0,utc_offset__-25200.0,utc_offset__-28800.0,utc_offset__-36000.0,utc_offset__-7200.0,utc_offset__10800.0,utc_offset__14400.0,utc_offset__19800.0,utc_offset__25200.0,utc_offset__28800.0,utc_offset__32400.0,utc_offset__3600.0,utc_offset__39600.0,utc_offset__7200.0,utc_offset__other,utc_offset__unk,location_clean,location_binary,location_public_binary,user_language,user_lang_ar,user_lang_de,user_lang_en,user_lang_en-gb,user_lang_es,user_lang_fr,user_lang_it,user_lang_ja,user_lang_ko,user_lang_nl,user_lang_other,user_lang_pt,user_lang_ru,user_lang_tr,profile_creation_timestamp,profile_creation_year,profile_creation_month,profile_creation_day,user_timezone,user_timezone_alaska,user_timezone_amsterdam,user_timezone_arizona,user_timezone_athens,user_timezone_atlantic time (canada),user_timezone_baghdad,user_timezone_bangkok,user_timezone_berlin,user_timezone_bern,user_timezone_bogota,user_timezone_brasilia,user_timezone_buenos aires,user_timezone_cairo,user_timezone_caracas,user_timezone_casablanca,user_timezone_central america,user_timezone_central time (us & canada),user_timezone_chennai,user_timezone_dublin,user_timezone_eastern time (us & canada),user_timezone_greenland,user_timezone_hawaii,user_timezone_istanbul,user_timezone_jakarta,user_timezone_kuala lumpur,user_timezone_kuwait,user_timezone_london,user_timezone_madrid,user_timezone_mexico city,user_timezone_mountain time (us & canada),user_timezone_mumbai,user_timezone_muscat,user_timezone_nairobi,user_timezone_new delhi,user_timezone_other,user_timezone_pacific time (us & canada),user_timezone_paris,user_timezone_pretoria,user_timezone_quito,user_timezone_riyadh,user_timezone_rome,user_timezone_santiago,user_timezone_seoul,user_timezone_sydney,user_timezone_tokyo,user_timezone_unk,user_timezone_volgograd,avg_daily_profile_visit_duration,num_followers_log,num_people_following_log,num_status_updates_log,num_direct_messages_log,avg_daily_profile_clicks_log,profile_category_business,profile_category_celebrity,profile_categ

## Define the Features that will be Included in Modeling

In [7]:
def selectFeatures(df):
    """Select features to be included in the models
    
    Args:
        df (DataFrame): Dataframe containing all variables to select from
    Return:
        features_cols (list): List of feature names to include in training dataset
    
    """
    # Select each set of one-hot encoded features 
    verification_status = [col for col in df.columns if ('verification_status_' in col)]
    profile_text_color = [col for col in df.columns if ('profile_text_color_' in col)]
    profile_page_color = [col for col in df.columns if ('profile_page_color_' in col)]
    profile_theme_color = [col for col in df.columns if ('profile_theme_color_' in col)]
    utc_offset = [col for col in df.columns if ('utc_offset_' in col)]
    numerical_norm = ['user_name_length', 'num_followers_log_norm', 'num_people_following_log_norm', 'num_status_updates_log_norm', 'num_direct_messages_log_norm', 'avg_daily_profile_visit_duration_norm', 'avg_daily_profile_clicks_log_norm']
    profile_category = [col for col in df.columns if ('profile_category_' in col)]
    user_timezone = [col for col in df.columns if ('user_timezone_' in col)]
    user_lang = [col for col in df.columns if ('user_lang_' in col)]
    profile_creation_norm = ['profile_creation_year_norm', 'profile_creation_month_norm', 'profile_creation_day_norm']
    binary = ['personal_url_binary', 'location_binary', 'location_public_binary', 'cover_image_binary', 'profile_view_size_custom']
    
    # Select the ones we want to use
    #features_cols = misc + verification_status + profile_text_color + profile_page_color + profile_theme_color + numerical + profile_category + profile_creation_norm + user_timezone + user_lang + utc_offset
    #features_cols = numerical_norm + binary + ['profile_creation_year_norm'] + verification_status + user_lang + profile_category 
    #features_cols = ['user_name_length_norm','num_followers_log_norm', 'num_people_following_log_norm', 'personal_url_binary', 'num_status_updates_log_norm', 'num_direct_messages_log_norm', 'location_public_binary', 'profile_creation_year_norm', 'avg_daily_profile_clicks_log_norm', 'avg_daily_profile_visit_duration_norm'] + verification_status + profile_category
    features_cols = numerical_norm + binary + verification_status + profile_category + profile_creation_norm + ['utc_offset_bins', 'user_timezone_bins', 'user_language_bins', 'user_name_length_bins']
    
    # Return list of features
    return features_cols

### Common Train and Test Features

In [8]:
# There might be features (after one-hot) that are in test but not in train, or vice-versa
common_features = list(set(selectFeatures(df_train)) & set(selectFeatures(df_test)))
discarded_features = np.setdiff1d(selectFeatures(df_train), selectFeatures(df_test))
print(f'Discarded features (in train but not in test or vice versa): {discarded_features}')

Discarded features (in train but not in test or vice versa): []


### Define Features and Target

In [9]:
# Select the features to consider in the feature selection, and define target variable
X_train = df_train[common_features]
X_test = df_test[common_features]
y_train = df_train['num_profile_likes_log']

### Feature Selection
We can use filter feature selection to measure the correlation of the features with regards to the target variable. Doing so, we can narrow down on a selected number of features and discard the ones with negligeable impact. The function `featureSelection` take as input an initial set of X_train, X_test and would return the same datasets but with `K` best features.

In [10]:
def featureSelection(X_train, X_test, y_train, scoring, bestK):
    """ Based on a correlation statistic, finds a subset of features to include in the datasets.
    
    Args:
        X_train (DataFrame): Training dataset with all features.
        X_test (DataFrame): Test dataset with all features.
        y_train (Series): Training target values.
        scoring (function): Correlation statistic tests. (ref: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)
        bestK (int): The k best features with highest scores to include in model ('all' to keep every features).
    Return:
        X_train_fs (DataFrame): Training dataset with subset of best features.
        X_test_fs (DataFrame): Test dataset with subset of best features.
        scores (DataFrame): All the features with corresponding scores.
    """
    
    # Feature extraction based on correlations with target
    fs = SelectKBest(score_func=scoring, k=bestK)
    fs.fit(X_train, y_train)
    
    # Keep only the best K features
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)

    # Create dataframe of features and scores
    scores = pd.DataFrame()
    for i in range(len(fs.scores_)):
        scores = scores.append(
            pd.DataFrame({'feature':[X_train.columns[i]], 'score':[fs.scores_[i]]}),
            ignore_index=True)
    scores = scores.sort_values('score', ascending=False).reset_index(drop=True)
    
    # Return train and test dataframes with subset of features, and all the scores
    return X_train_fs, X_test_fs, scores

In [11]:
# Select subset of features to include in the models
bestK = 20
X_train_fs, X_test_fs, scores = featureSelection(X_train, X_test, y_train, f_regression, bestK)

In [12]:
# All the feature swith their scores
scores.head(bestK)

,feature,score
0,personal_url_binary,2764.659495
1,num_people_following_log_norm,1541.779581
2,num_status_updates_log_norm,1383.506422
3,user_timezone_bins,766.743311
4,utc_offset_bins,607.574172
5,location_public_binary,566.390249
6,profile_category_unknown,522.044102
7,user_language_bins,451.375416
8,avg_daily_profile_clicks_log_norm,376.983910
9,verification_status_verified,321.061044


## Single Runs
Fit and evaluate models on a single training-validation split for quick tests. The `evaluate` function simply calculates the performance of a model by using RMSE or RMSLE. It can be used in both single runs and during cross-validation. The `singleRun` function splits the training set into training and validation for model fitting and evaluation. As the name implies, the model is only evaluated on one specific split of the training dataset.

In [13]:
def evaluate(predictions, y_test, metric):
    """ Function to evaluate the performance of model based on different metrics.
    
    Args:
        model (model): Classifier or regressor model to use in the prediction.
        y_test (ndarray): The training target values.
        test (string): The type of test (rmse or rmsle).
    Return:
        score (float): The average score.
    """
    
    # Calculate score
    if metric == 'rmse':
        score = np.sqrt(np.square(predictions - y_test).mean())
    elif metric == 'rmsle':
        score = np.sqrt(np.square(np.log(predictions + 1) - np.log(y_test + 1)).mean())
    else:
        score = 'Error'
    
    print(f'Score ({metric.upper()}): {score:.3f}')
    
    return score

In [14]:
def singleRun(model, features, target):
    """ Function evaluates a model on a single train/validation split.
    
    Args:
        model (model): Classifier or regressor model to use in the prediction
        features (ndarray): The training features.
        target (ndarray): The training target values.
    """
    # Split df_train into training and validation datasets
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, 
                                                          test_size=0.25, shuffle=True, random_state=42)
    
    # Fit model using training set
    model.fit(X_train, y_train)
    
    # Predict using fitted model on test set
    predictions = model.predict(X_valid)
    
    # Evaluate model using validation set
    evaluate(predictions, y_valid, 'rmse')

## Regressors
Here we can quickly test different regressor without any hyperparameter tuning just to have an overall idea of the models' baseline performance.

### Ridge Regression

In [15]:
lmbr = Ridge()
singleRun(lmbr, X_train_fs, y_train)

Score (RMSE): 1.796


### Poisson Regression

In [16]:
pr = PoissonRegressor()
singleRun(pr, X_train_fs, y_train)

Score (RMSE): 2.026


### SVR

In [17]:
svr = SVR()
singleRun(svr, X_train_fs, y_train)

Score (RMSE): 1.869


### Decision Tree

In [18]:
dt = DecisionTreeRegressor()
singleRun(dt, X_train_fs, y_train)

Score (RMSE): 2.477


### Random Forest

In [19]:
rf = RandomForestRegressor()
singleRun(rf, X_train_fs, y_train)

Score (RMSE): 1.769


### AdaBoost

In [20]:
ab = AdaBoostRegressor()
singleRun(ab, X_train_fs, y_train)

Score (RMSE): 2.039


### Bagging

In [21]:
br = BaggingRegressor()
singleRun(br, X_train_fs, y_train)

Score (RMSE): 1.838


### XGBoost

In [22]:
xgb = XGBRegressor()
singleRun(xgb, X_train_fs, y_train)

Score (RMSE): 1.809


## Cross-Validation
Now, instead of doing a single test of the models, we can do k-fold Cross-Validation. The function `KFoldCV` fits a model using the training set and evaluate the model's performance `K` times, each time using a different validation set. Of course, we can also use `cross_val_score()` to accomplish the same thing. 

For pros and cons, ref: https://stackoverflow.com/questions/60938685/what-is-the-difference-between-getting-score-value-from-kfold-fit-score-vs-using#:~:text=A%20limitation%20of%20cross_val_score%20is,necessary%20to%20use%20for%20%2B%20kfold%20.

In [23]:
def KFoldCV(model, k, X, y, metric):
    """ Custom function to carry out K-Fold Cross-Validation on a training dataset manually.
    It allows for more flexibility as compared to cross_val_score().
    Args:
        model (model): Model used for prediction.
        k (int): The number of folds in CV.
        X (DataFrame/ndarray): Training dataset.
        y (Series/ndarray): Training target values.
        metric (string): The metric used to measure model performance ('rmse' or 'rmsle').
    """
    # K-Fold CV
    kf = KFold(n_splits=k)
    # Initialize list for scores
    scores = []
    # Ensure that X and y are ndarray
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
    if isinstance(y, pd.Series):
        y = y.to_numpy()
        
    print('Manual K-Fold Cross-Validation:')
    
    # Carry out K-Fold CV
    for train_index, valid_index in kf.split(X):
        # Split data
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        # Fit model on training set
        model.fit(X_train, y_train)
        # Predict with validation set
        predictions = model.predict(X_valid)
        # Evaluate model
        scores.append(evaluate(predictions, y_valid, metric))
    
    scores = np.array(scores)
    print(f'Mean Accuracy: {scores.mean():0.3f} (+/- {scores.std() * 2:0.3f})')
    print('========================================')

In [24]:
def sklearnKFoldCV(model, k, X, y, metric):
    
    if metric == 'rmse':
        scoring_metric = 'neg_root_mean_squared_error'
        
    print('Sklearn K-Fold Cross-Validation:')
        
    cv=KFold(n_splits=k, shuffle=True, random_state=42)
    scores = abs(cross_val_score(model, X, y, scoring=scoring_metric, cv=cv))
    _ = [print(f'Score (RMSE): {score:.3f}') for score in scores]
    print(f'Mean Accuracy: {scores.mean():0.3f} (+/- {scores.std() * 2:0.3f})')

### Ridge Regression

In [25]:
lmbr = Ridge(alpha=1, solver='saga')
KFoldCV(lmbr, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(lmbr, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.784
Score (RMSE): 1.806
Score (RMSE): 1.751
Score (RMSE): 1.805
Score (RMSE): 1.917
Mean Accuracy: 1.812 (+/- 0.112)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 1.825
Score (RMSE): 1.777
Score (RMSE): 1.870
Score (RMSE): 1.753
Score (RMSE): 1.846
Mean Accuracy: 1.814 (+/- 0.086)


### Poisson Regression

In [26]:
pr = PoissonRegressor(fit_intercept=True, max_iter=500)
KFoldCV(pr, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(pr, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.919
Score (RMSE): 2.015
Score (RMSE): 1.979
Score (RMSE): 2.009
Score (RMSE): 2.089
Mean Accuracy: 2.002 (+/- 0.110)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 2.051
Score (RMSE): 1.982
Score (RMSE): 2.066
Score (RMSE): 1.911
Score (RMSE): 2.000
Mean Accuracy: 2.002 (+/- 0.110)


### SVR

In [27]:
svr = SVR(kernel='poly', degree=4, C=15, coef0=1.5)
KFoldCV(svr, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(svr, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.764
Score (RMSE): 1.754
Score (RMSE): 1.707
Score (RMSE): 1.750
Score (RMSE): 1.853
Mean Accuracy: 1.766 (+/- 0.096)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 1.756
Score (RMSE): 1.738
Score (RMSE): 1.820
Score (RMSE): 1.703
Score (RMSE): 1.796
Mean Accuracy: 1.763 (+/- 0.083)


In [28]:
# Fit model and predict
svr.fit(X_train_fs, y_train)
svr_predictions = svr.predict(X_test_fs)

### Decision Tree

In [29]:
dt = DecisionTreeRegressor(criterion='mse', max_depth=10, min_samples_split=2)
KFoldCV(dt, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(dt, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 2.032
Score (RMSE): 2.090
Score (RMSE): 2.011
Score (RMSE): 2.155
Score (RMSE): 2.078
Mean Accuracy: 2.073 (+/- 0.100)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 2.079
Score (RMSE): 2.154
Score (RMSE): 2.092
Score (RMSE): 1.947
Score (RMSE): 2.060
Mean Accuracy: 2.066 (+/- 0.135)


### Random Forest

In [30]:
rf = RandomForestRegressor(n_estimators=3, max_depth=20, min_samples_split=3, max_features='log2')
KFoldCV(rf, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(rf, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.975
Score (RMSE): 2.019
Score (RMSE): 1.922
Score (RMSE): 1.981
Score (RMSE): 2.042
Mean Accuracy: 1.988 (+/- 0.082)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 2.018
Score (RMSE): 1.976
Score (RMSE): 2.033
Score (RMSE): 1.942
Score (RMSE): 2.031
Mean Accuracy: 2.000 (+/- 0.071)


### AdaBoost

In [31]:
ab = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=7, min_samples_leaf=25),
                       n_estimators=500, learning_rate=0.01, loss='linear')
KFoldCV(ab, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(ab, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.725
Score (RMSE): 1.748
Score (RMSE): 1.696
Score (RMSE): 1.748
Score (RMSE): 1.824
Mean Accuracy: 1.748 (+/- 0.085)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 1.795
Score (RMSE): 1.704
Score (RMSE): 1.778
Score (RMSE): 1.685
Score (RMSE): 1.790
Mean Accuracy: 1.750 (+/- 0.093)


In [32]:
# Fit model and predict
ab.fit(X_train_fs, y_train)
ab_predictions = ab.predict(X_test_fs)

### Bagging

In [33]:
br = BaggingRegressor()
KFoldCV(br, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(br, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.808
Score (RMSE): 1.837
Score (RMSE): 1.797
Score (RMSE): 1.835
Score (RMSE): 1.902
Mean Accuracy: 1.836 (+/- 0.073)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 1.883
Score (RMSE): 1.805
Score (RMSE): 1.848
Score (RMSE): 1.786
Score (RMSE): 1.867
Mean Accuracy: 1.838 (+/- 0.074)


### XGBoost

In [34]:
xgb = XGBRegressor(subsample=0.7, max_depth=3, n_estimators=25, objective='count:poisson', verbosity=1)
KFoldCV(xgb, 5, X_train_fs, y_train, 'rmse')
sklearnKFoldCV(xgb, 5, X_train_fs, y_train, 'rmse')

Manual K-Fold Cross-Validation:
Score (RMSE): 1.767
Score (RMSE): 1.797
Score (RMSE): 1.770
Score (RMSE): 1.773
Score (RMSE): 1.863
Mean Accuracy: 1.794 (+/- 0.072)
Sklearn K-Fold Cross-Validation:
Score (RMSE): 1.822
Score (RMSE): 1.749
Score (RMSE): 1.821
Score (RMSE): 1.726
Score (RMSE): 1.820
Mean Accuracy: 1.788 (+/- 0.083)


## Prepare Submission File

In [35]:
# Choose model and transform predictions
predictions = svr_predictions
predictions = np.exp(predictions) - 1
submission = pd.DataFrame({'Id':df_test['Id'], 'Predicted':predictions})

In [36]:
submission

,Id,Predicted
0,49I3SOKLI2CMNGP4,2506.916613
1,727IRIR59A3P88LK,3453.149283
2,LN95SD15SRPCEE8F,54.826055
3,TB11I7F0PN033D4T,2640.461249
4,32PSGCK5PATHMR07,110.090628
...,...,...
2495,7SDJE48EFRPPNEJK,98.276235
2496,PRT8RDNG6E86518P,5012.459819
2497,SRNIBIK27BQ2M3PB,4715.141227
2498,6CP232J9R8N84702,915.006561


### Export Submission File

In [37]:
submission.to_csv('submission_SVR_08-2.csv', index=False)